# **Importing Libraries**

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 4.2 MB 40.0 MB/s 
     |████████████████████████████████| 1.2 MB 45.8 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 6.6 MB 43.9 MB/s 
     |████████████████████████████████| 596 kB 42.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=f8ba024c447a1aed7190e1fecf92067d6a596af84c0e45e2c7a6e5cf7a0e6071
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:

from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [ ]:
import nltk
import re

from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Loading target dataset**

In [ ]:
# Load test data
import json
with open('accesslog.json') as in_file:
    data = json.load(in_file)

In [ ]:
tags = []
for i in range(len(data)):
  tags.append(data[i]['_source']['tags'])

In [ ]:
df1 = pd.DataFrame()
df1['tags'] = tags

In [ ]:
df1

,tags
0,[Possible SQL Injection]
1,[]
2,[Possible SQL Injection]
3,"[Possible SQL Injection, Possible Cross Site S..."
4,[]
...,...
82,[4XX Client Errors]
83,[]
84,"[Possible Cross Site Scripting, 4XX Client Err..."
85,"[Possible Cross Site Scripting, 4XX Client Err..."


# **Loading Training dataset**

In [ ]:
df = pd.read_csv('Alert_file2.csv')
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df['message'] = df['message'].apply(lambda x: x.lower())

In [ ]:
df['tags'] = df1['tags']

In [ ]:
display(df)

,message,source_ip,tags
0,102281393 16aug2021114727 0530 get union http1...,51.222.253.11,[Possible SQL Injection]
1,102281393 16aug2021114727 0530 get faviconico ...,10.228.0.95,[]
2,102281393 16aug2021114732 0530 get 20select 20...,10.208.0.229,[Possible SQL Injection]
3,102281393 16aug2021112839 0530 get unionselect...,10.208.0.229,"[Possible SQL Injection, Possible Cross Site S..."
4,102281393 16aug2021112843 0530 get waitfor 20d...,10.228.0.95,[]
...,...,...,...
82,102281394 17aug2021162151 0530 get faviconico ...,114.55.9.245,[4XX Client Errors]
83,102281394 17aug2021162154 0530 get http11 200 ...,10.228.0.95,[]
84,102281394 17aug2021162205 0530 get alert http1...,10.228.0.95,"[Possible Cross Site Scripting, 4XX Client Err..."
85,102281394 17aug2021162217 0530 get 3cscript 3e...,114.55.9.245,"[Possible Cross Site Scripting, 4XX Client Err..."


In [ ]:
documents = df['message']

# **Data preprocessing**

In [ ]:
def preprocessing(input_documents):
     wordnet_lemmatizer = WordNetLemmatizer()
     punctuations = set('''!()-[]{};:'"\,<>.`/?``#$%^&*_~''')                         
    
     docs = []
     for sentence in input_documents:
         sentence = sentence.lower()
         string_list = []
         sentence_words = nltk.word_tokenize(sentence)
         for word in sentence_words:
             
             lemmatized_word  =  wordnet_lemmatizer.lemmatize(word)
             
             lemmatized_word = re.sub(r"[^0-9A-Za-z]+", "", lemmatized_word)             
             lemmatized_word = re.sub(r"-", "", lemmatized_word)
             if lemmatized_word not in stop_words and punctuations:
                 string_list.append(lemmatized_word)
         final_word = " ".join(string_list)
         final_word = re.sub("[\t ]{2,}", " ", final_word)
   
         docs.append(final_word)
         
     return docs

In [ ]:
 documents = preprocessing(documents)
 print(f'{len(documents)} documents')

87 documents


In [ ]:
df = df.drop('message', axis = 1)

In [ ]:
df['message'] = documents

In [ ]:
# Printing unique source-IPs

import numpy as np
np.unique(df['source_ip'])

array(['10.208.0.229', '10.228.0.93', '10.228.0.95', '10.228.10.59',
       '10.228.10.75', '10.228.11.121', '11.208.0.92', '114.55.9.245',
       '195.154.123.130', '51.222.253.11', '54.36.148.129'], dtype=object)

In [ ]:
# Removal of datapoints with missing important features

df = df.dropna()

In [ ]:
def remove_empty_list(tag):
  if tag == []:
    return 'NaN'
  return tag[0]

df['tags'] = df['tags'].apply(lambda tag: remove_empty_list(tag))

In [ ]:
df = df[df['tags'] != 'NaN']

In [ ]:
df = df.drop_duplicates()

In [ ]:
# Splitting dataset into train and test.......

X_train, X_test, y_train, y_test = train_test_split(df.message, df.Label, test_size=0.2, random_state=42)

In [ ]:
# Relevant Query string for detcting sql injection........

query_string = ['waitfor','waitfordelay','union select','union all select', 'order by', 'you have an error', 'unknown column', 'unionselect', 'orderby', 'order by 1', '@@version','@@user', '@@database']


# **Model Training**

In [ ]:
import sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
# Embedding..........

df['embedded_message'] = df['message'].apply(lambda sentences: model.encode(sentences.lower()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df['embedded_tags'] = df['tags'].apply(lambda sentences: model.encode(sentences.lower()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
query_string = preprocessing(query_string)

In [ ]:
query_string_embeddings = model.encode(query_string)

# **Applying Cosine Similarity**

In [ ]:
def similarity_check(sent1,query):
  return round(max(max(cosine_similarity([sent1], query))),2)

In [ ]:
df['Similarity_score'] = df['embedded_message'].apply(lambda sent: similarity_check(sent,query_string_embeddings[:]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Similarity_list = df['Similarity_score'].values

In [ ]:
Similarity_list

array([0.26, 0.25, 0.47, 0.24, 0.32, 0.24, 0.24, 0.29, 0.28, 0.33, 0.29,
       0.33, 0.25, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26,
       0.26, 0.25, 0.26, 0.17, 0.18, 0.28, 0.28, 0.32, 0.31, 0.31, 0.17,
       0.3 , 0.27, 0.28, 0.31, 0.25, 0.24, 0.24, 0.25, 0.29, 0.25, 0.18,
       0.17, 0.31, 0.33, 0.26, 0.3 , 0.29, 0.29, 0.39, 0.32, 0.19, 0.27,
       0.19, 0.28, 0.17, 0.27, 0.18, 0.22, 0.28, 0.27, 0.22, 0.22, 0.29,
       0.26, 0.26, 0.3 , 0.33, 0.2 , 0.43, 0.33, 0.33, 0.18, 0.24, 0.36,
       0.26], dtype=float32)

In [ ]:
import statistics
import numpy as np
Similarity_mean = statistics.mean(Similarity_list)
Similarity_standard_deviation = np.std(Similarity_list)
alpha = 0.1875
threshold = Similarity_mean + alpha * Similarity_standard_deviation
print(Similarity_mean)
print(Similarity_standard_deviation)
print(threshold)

0.26884615
0.05502868
0.2791640318464488


In [ ]:
result = df[df['Similarity_score'] >= 0.28]

In [ ]:
result = result.sort_values('Similarity_score',  ascending=False)

In [ ]:
final_res = result[['source_ip', 'message', 'Similarity_score']]

In [ ]:
print("Most relevant SQL injection alert from the available dataset: ")
print(final_res.head(10))

Most relevant SQL injection alert from the available dataset: 
          source_ip                                            message  \
3      10.208.0.229  102281393 16aug2021112839 0530 get unionselect...   
79      10.228.0.95  102281394 17aug2021162156 0530 get admin http1...   
59      10.228.0.95  102281393 16aug2021161415 0530 get admin http1...   
85     114.55.9.245  102281394 17aug2021162217 0530 get 3cscript 3e...   
77      10.228.0.95  1022810118 17aug2021160122 0530 get union 20al...   
54    54.36.148.129  102281393 16aug2021163951 0530 get union http1...   
12     10.228.10.59  102281393 16aug2021114005 0530 get union http1...   
14     10.228.10.75  102281393 16aug2021113713 0530 get union http1...   
81  195.154.123.130  102281394 17aug2021162151 0530 get union http1...   
80     114.55.9.245  102281394 17aug2021162200 0530 get sbadmin htt...   

    Similarity_score  
3               0.47  
79              0.43  
59              0.39  
85              0.36  
77     

In [ ]:
test_data_accuracy = round(df['tags'].eq(result['tags']).mean(),2)*100

In [ ]:
print("Accuracy of the SQL Injection alert detection: ", test_data_accuracy)

Accuracy of the SQL Injection alert detection:  74.0
